# Segmentating and Clustering neighbourhoods of Toronto


### This is the first part of the Notebook 

***Scraping table from a webpage***

In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests


In [18]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


distutils: c:\users\asus\appdata\local\programs\python\python38\Include\UNKNOWN
sysconfig: c:\users\asus\appdata\local\programs\python\python38\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\asus\appdata\local\programs\python\python38\Include\UNKNOWN
sysconfig: c:\users\asus\appdata\local\programs\python\python38\Include
user = False
home = None
root = None
prefix = None


In [19]:
page=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(page,'html.parser')
print(soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [30]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [31]:
print(table_contents[:4])

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}]


In [33]:
df=pd.DataFrame(table_contents)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [34]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [35]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## This is the second part of the Notebook

## In this part we will retrieve the longitude and latitude of postal codes

In [41]:
import geocoder


In [50]:
lat_lon=pd.read_csv('C:/Users/Asus/Downloads/Geospatial_Coordinates.csv')


In [51]:
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [52]:
df_lat=pd.concat([df,lat_lon],axis=1,join="inner")
df_lat.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,M1H,43.773136,-79.239476


In [53]:
del df_lat["Postal Code"]
df_lat.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476


# This is the third part of the notebook

## In this part we will cluster and examine the neighborhoods of Toronto

In [54]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [55]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_lat['Borough'].unique()),
        df_lat.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


In [56]:
from geopy.geocoders import Nominatim

In [57]:
df1=df_lat[df_lat['Borough'].str.contains('Toronto'),regex=False]
df1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
15,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
19,M4E,East Toronto,The Beaches,43.786947,-79.385975
20,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714


# Plotting above neighborhoods on map using folium 

In [59]:
address='Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [61]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,long,borough,neighborhood in zip(df1['Latitude'],df1['Longitude'],df1['Borough'],df1['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3186cc',
                       fill_opacity=0.7,
                       parse_html=False).add_to(map_toronto)
map_toronto

## Let us explore the first neighborhood in the dataframe df1

In [67]:
df1=df1.reset_index(drop=True)
df1.head()

,level_0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,3,M4E,East Toronto,The Beaches,43.786947,-79.385975
4,4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714


In [68]:
del df1['level_0']

In [94]:
df1

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M4E,East Toronto,The Beaches,43.786947,-79.385975
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
5,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
6,M6G,Downtown Toronto,Christie,43.753259,-79.329656
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944
9,M4J,East York/East Toronto,The Danforth East,43.706397,-79.309937


In [70]:
df1.loc[0,'Neighborhood']

'Regent Park, Harbourfront'

In [71]:
n_lat=df1.loc[0,'Latitude']
n_lon=df1.loc[0,'Longitude']
n_name=df1.loc[0,'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(n_name, 
                                                               n_lat, 
                                                               n_lon))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


### Foursquare Credentials

In [72]:
CLIENT_ID = 'WY04EP2PLIS1BOQISJXQLPVHLET2COE2SWVZ05N4WMNJDQIU' 
CLIENT_SECRET = 'QTAFWTQFFLWA30U2KNGQHSNC11KCIKMRKAOQ3OK2MMUFIJ0X' 
VERSION = '20180605' 
LIMIT = 100 



### Exploring venues

In [75]:
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
n_lat,
n_lon,
radius,
LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=WY04EP2PLIS1BOQISJXQLPVHLET2COE2SWVZ05N4WMNJDQIU&client_secret=QTAFWTQFFLWA30U2KNGQHSNC11KCIKMRKAOQ3OK2MMUFIJ0X&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100'

In [76]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '608ab42d7d1aa64e2accb9bb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4beee041e24d20a1cd857314',
       'name': 'RBC Royal Bank',
       'location': {'address': '4374 KINGSTON RD',
        'crossStreet': 'Kingston & Lawrence',
        'lat': 43.76678992471017,
        'lng': -79.19115118872593,
        '

In [77]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [78]:
from pandas.io.json import json_normalize

In [87]:
venues=results['response']['groups'][0]['items']
nearby_venues=pd.json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [88]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Sail Sushi,Restaurant,43.765951,-79.191275
3,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406


In [89]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


# Clustering the Neighborhood

In [96]:
k=4
df1_clustering=df1.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(df1_clustering)
kmeans.labels_[0:10]

array([3, 3, 2, 2, 2, 2, 3, 2, 0, 3])

In [98]:
df1.insert(0,'Cluster labels',kmeans.labels_)
df1

,Cluster labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,3,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,2,M4E,East Toronto,The Beaches,43.786947,-79.385975
4,2,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
5,2,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
6,3,M6G,Downtown Toronto,Christie,43.753259,-79.329656
7,2,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
8,0,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944
9,3,M4J,East York/East Toronto,The Danforth East,43.706397,-79.309937


In [102]:
map_cluster=folium.Map(location=[latitude,longitude],zoom_start=11)
x=np.arange(k)

ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood'], df1['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster
                                )
map_cluster

## Thank You